# <center>Comparison of rent and sale prices</center>

## 0) Imports

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import glob

import dataprocessing as dp

%load_ext nb_black
%matplotlib inline

plt.rcParams["figure.figsize"] = (15, 10)
pd.set_option("display.precision", 2)

<IPython.core.display.Javascript object>

## 1) Data Engineering

#### find dates:

In [2]:
dates = sorted(
    [
        s[s.find("mieten") + len("mieten") : s.rfind(".csv")]
        for s in glob.glob("../data/mieten*")
    ]
)

<IPython.core.display.Javascript object>

#### load data:

In [3]:
rent_df = dp.create_rent_df(dates[-1])
sale_df = dp.create_sale_df(dates[-1])

<IPython.core.display.Javascript object>

#### add links:

In [4]:
rent_df["link"] = "https://www.immobilienscout24.de/expose/" + rent_df[
    "scoutId"
].astype(str)
sale_df["link"] = "https://www.immobilienscout24.de/expose/" + sale_df[
    "scoutId"
].astype(str)

<IPython.core.display.Javascript object>

#### add return of investment based on median rents for zipCode

In [5]:
g = rent_df.groupby("zipCode")["rent_m2"].median()
g.name = "rent_m2_zipCode"
sale_df = sale_df.join(g, on="zipCode").copy()

sale_df["yearly_ROI"] = sale_df["rent_m2_zipCode"] * 12 / sale_df["price_m2"]

<IPython.core.display.Javascript object>

#### add combined dataframe with median rents and prices:

In [6]:
group = "zipCode"

combined_df = pd.concat(
    [
        rent_df.groupby(group)["rent_m2"].median(),
        rent_df.groupby(group)["rent_m2"].size(),
        sale_df.groupby(group)["price_m2"].median(),
        sale_df.groupby(group)["price_m2"].size(),
        sale_df.groupby(group)["yearly_ROI"].median(),
    ],
    axis=1,
    keys=[
        "median_rent_m2",
        "num_rent",
        "median_price_m2",
        "num_sale",
        "median_yearly_ROI",
    ],
).copy()

combined_df = combined_df.replace([np.inf, -np.inf], np.nan)
combined_df = combined_df.dropna()

<IPython.core.display.Javascript object>

#### select features:

In [7]:
rent_select_columns = [
    "rent_m2",
    "baseRent",
    "livingSpace",
    "noRooms",
    "zipCode",
    "regio2",
    "link",
]
rent_select_df = rent_df[rent_select_columns].copy()

sale_select_columns = [
    "yearly_ROI",
    "price_m2",
    "purchasePrice",
    "livingSpace",
    "noRooms",
    "zipCode",
    "regio2",
    "link",
]
sale_select_df = sale_df[sale_select_columns].copy()

<IPython.core.display.Javascript object>

## 2) Results

### 2.0) Apartments by region

#### regions overview:

In [8]:
min_num = 5
combined_filt_ord = "median_yearly_ROI"

combined_filt = (combined_df["num_rent"] > min_num) & (
    combined_df["num_sale"] > min_num
)
combined_df[combined_filt].sort_values(combined_filt_ord, ascending=False)

,median_rent_m2,num_rent,median_price_m2,num_sale,median_yearly_ROI
zipCode,,,,,
4639,5.00,6.0,25.37,7.0,2.36
47169,6.23,53.0,142.25,22.0,0.53
6231,5.34,101.0,421.70,6.0,0.15
6369,4.50,10.0,385.59,8.0,0.14
47229,6.42,25.0,576.92,9.0,0.13
...,...,...,...,...,...
22085,19.65,12.0,12154.39,26.0,0.02
18609,9.14,6.0,5952.38,57.0,0.02
83700,16.86,12.0,11227.83,28.0,0.02


<IPython.core.display.Javascript object>

#### single region:

In [9]:
combined_arg = 4639
combined_df.loc[combined_arg]

median_rent_m2        5.00
num_rent              6.00
median_price_m2      25.37
num_sale              7.00
median_yearly_ROI     2.36
Name: 4639, dtype: float64

<IPython.core.display.Javascript object>

### 2.1) Apartments for rent

In [10]:
rent_filt_col = "zipCode"
rent_filt_arg = [4639]
rent_ord = "rent_m2"

rent_filt = rent_select_df[rent_filt_col].isin(rent_filt_arg)
rent_select_df[rent_filt].sort_values(rent_ord).head(20)

,rent_m2,baseRent,livingSpace,noRooms,zipCode,regio2,link
81245,4.48,280.00,62.53,2.0,4639,Altenburger_Land_Kreis,https://www.immobilienscout24.de/expose/113589133
85089,4.53,281.00,62.00,2.0,4639,Altenburger_Land_Kreis,https://www.immobilienscout24.de/expose/110498274
64739,5.00,472.05,94.41,4.0,4639,Altenburger_Land_Kreis,https://www.immobilienscout24.de/expose/117006387
67112,5.00,270.00,54.00,2.0,4639,Altenburger_Land_Kreis,https://www.immobilienscout24.de/expose/116777942
86304,5.10,250.00,49.00,2.0,4639,Altenburger_Land_Kreis,https://www.immobilienscout24.de/expose/109163868
46887,5.27,300.00,56.92,3.0,4639,Altenburger_Land_Kreis,https://www.immobilienscout24.de/expose/118566569


<IPython.core.display.Javascript object>

### 2.2) Apartments for sale

In [11]:
sale_filt_col = "zipCode"
sale_filt_arg = [4639]
sale_ord = "yearly_ROI"

sale_filt = sale_select_df[sale_filt_col].isin(sale_filt_arg)
sale_select_df[sale_filt].sort_values(sale_ord, ascending=False).head(50)

,yearly_ROI,price_m2,purchasePrice,livingSpace,noRooms,zipCode,regio2,link
32276,4.77,12.58,4000.0,318.0,10.0,4639,Altenburger_Land_Kreis,https://www.immobilienscout24.de/expose/115369915
6462,2.44,24.59,1500.0,61.0,2.0,4639,Altenburger_Land_Kreis,https://www.immobilienscout24.de/expose/118724476
6468,2.44,24.64,1700.0,69.0,2.0,4639,Altenburger_Land_Kreis,https://www.immobilienscout24.de/expose/118724450
6474,2.36,25.37,1700.0,67.0,2.0,4639,Altenburger_Land_Kreis,https://www.immobilienscout24.de/expose/118724459
6472,2.36,25.42,1500.0,59.0,2.0,4639,Altenburger_Land_Kreis,https://www.immobilienscout24.de/expose/118724441
6461,2.33,25.81,1600.0,62.0,2.0,4639,Altenburger_Land_Kreis,https://www.immobilienscout24.de/expose/118724472
36091,0.06,937.50,37500.0,40.0,1.0,4639,Altenburger_Land_Kreis,https://www.immobilienscout24.de/expose/103586059


<IPython.core.display.Javascript object>